In [2]:
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [3]:
filename = "text8.zip"
def read_data(filename):
  with zipfile.ZipFile(filename) as f:
    name = f.namelist()[0]
    data = tf.compat.as_str(f.read(name))
  return data
  
text = read_data(filename)
print('text size %d' % len(text))

text size 100000000


In [4]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


In [5]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0

def nchar2id(string):
    nid = np.zeros(shape=[len(string)])
    j = 0
    for i in string:
        #print(i)
        nid[j] = char2id(i)
        j = j + 1
    return nid

def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '
print(nchar2id('a zz'), nchar2id('z'))
print(nchar2id('a'), nchar2id('z'), char2id(' '), char2id('i'))
#print(id2char(1), id2char(26), id2char(0))

[  1.   0.  26.  26.] [ 26.]
[ 1.] [ 26.] 0 9


In [14]:
batch_size=32
num_unrollings=10
num_ch = 1
num_nodes = 40  #网络有多少节点
bi_voc = vocabulary_size * vocabulary_size
embedding_size = 15 
class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]#把数据切成batch_size块，每次取~
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    #batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    batch = np.zeros(shape=(self._batch_size), dtype=np.int32)#两个单词用一个词向量表示
    for b in range(self._batch_size):
      ch1 = char2id(self._text[self._cursor[b]])
      if self._text_size-1 == self._cursor[b]:
        ch2 = 0
      else:
        ch2 = char2id(self._text[self._cursor[b]+1])
      batch[b] = ch1*vocabulary_size + ch2
      self._cursor[b] = (self._cursor[b] + 2) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def bigram(char):
    s = []
    for ch in char:
      s.append(id2char(ch//vocabulary_size)+id2char(ch%vocabulary_size))
    return s

def bigrams(encodings):
  return [bi2str(e) for e in encodings]


def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  #batch.shape[0]:num_unrollings;batch.shape[1]:batch_size;batch.shape[2]:num_ch
  #s = [''] * batches[0].shape[0]
  s = ['']* batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, bigram(b))]
  return s

def bi2str(encoding):
    return id2char(encoding // vocabulary_size ) + id2char(encoding % vocabulary_size)

def bibatches2string(batches):
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, bigrams(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

bi_onehot = np.zeros((bi_voc, bi_voc))
np.fill_diagonal(bi_onehot, 1)
def bi2onehot(encodings):
    return [bi_onehot[e] for e in encodings]

# print(batches2string(train_batches.next()))
# print(batches2string(train_batches.next()))
# print(batches2string(valid_batches.next()))
# b =valid_batches.next()
# print(batches2string(b))
# print(b)
# #train_batches.next()
# a = train_batches.next()
# print(a)
# print('...',a[0])

In [7]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction,size = vocabulary_size):
  """Turn a (column) prediction into 1-hot encoded samples."""
  #p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p = np.zeros(shape=[1, size], dtype=np.float32)
  p[0, sample_distribution(prediction[0])] = 1
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, embedding_size])
  return b/np.sum(b, 1)[:,None]

def random_distribution(size=vocabulary_size):
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, size])
  return b/np.sum(b, 1)[:,None]

In [8]:
graph = tf.Graph()
with graph.as_default():
  
  gate_count = 4
  # Parameters:
  input_weights = tf.Variable(tf.truncated_normal([embedding_size, num_nodes*gate_count], -0.1, 0.1))
  output_weights = tf.Variable(tf.truncated_normal([num_nodes, num_nodes*gate_count], -0.1, 0.1))
  bias = tf.Variable(tf.zeros([1, num_nodes*gate_count]))
    
  embeddings = tf.Variable(tf.random_uniform([bi_voc, embedding_size], -1.0, 1.0))
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, bi_voc], -0.1, 0.1))
  b = tf.Variable(tf.zeros([bi_voc]))
  np_embeds = np.zeros((bi_voc, bi_voc))
  bigramonehot = tf.constant(np.reshape(np_embeds, -1), dtype=tf.float32, shape=[bi_voc, bi_voc])
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    values = tf.split(1, gate_count, tf.sigmoid(tf.matmul(i, input_weights) + tf.matmul(o, output_weights) + bias))
    input_gate = values[0]
    forget_gate = values[1]
    update = values[2]
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = values[3]
    return output_gate * tf.tanh(state), state

  # Input data.
  tf_train_data = tf.placeholder(tf.int32, shape=[num_unrollings + 1, batch_size], name='tf_train_data')
  train_data = list()
  for i in tf.split(0, num_unrollings + 1, tf_train_data):
    train_data.append(tf.squeeze(i))
  train_inputs = train_data[:num_unrollings]

  embed_inputs = tf.nn.embedding_lookup(embeddings,train_inputs)
  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in range(embed_inputs.get_shape()[0]):
    output, state = lstm_cell(embed_inputs[i], output, state)
    outputs.append(output)
  
  embed_labels = list()
  for i in train_data[1:]:
    embed_labels.append(tf.gather(bigramonehot,i))
  embed_labels = tf.concat(0,embed_labels)
  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0,outputs), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=embed_labels, logits=logits))

  print(embed_labels)
  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 500, 0.9, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.int32, shape=[1])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  embed_sample_input = tf.nn.embedding_lookup(embeddings, sample_input)
  sample_output, sample_state = lstm_cell(
    embed_sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

Tensor("concat:0", shape=(320, 729), dtype=float32)


In [9]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    _, mloss, lr, predictions = session.run([optimizer, loss, learning_rate, train_prediction], 
                                            feed_dict={tf_train_data: batches})
    mean_loss += mloss
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print('Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = list(batches)[1:]
      labels = np.concatenate([bi2onehot(j) for j in labels])
      print('Minibatch perplexity: %.2f' % float(np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = np.argmax(sample(random_distribution(bi_voc),bi_voc))
          sentence = bi2str(feed)
          #sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(49):
            prediction = sample_prediction.eval({sample_input: [feed]})
            feed = np.argmax(sample(prediction,bi_voc))
            #sentence += characters(feed)[0]
            sentence += bi2str(feed)
          print(sentence,'++')
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
#       valid_logprob = 0
#       for _ in range(valid_size):
#         b = valid_batches.next()
#         predictions = sample_prediction.eval({sample_input: b[0]})
#         valid_logprob = valid_logprob + logprob(predictions, b[1][0])
#       print('Validation set perplexity: %.2f' % float(np.exp(
#         valid_logprob / valid_size)))

Initialized
Average loss at step 0: 0.000000 learning rate: 10.000000
Minibatch perplexity: 725.45
rtdiqrwxnnlkukrufnsrmzr bljtspqhzdlrxgrebgzzhyrrx oygbnuxhuclwpdsnrkqnub eawgshxbwxyjzhsqgrvfbiixgzd ++
lrnntptzumfkfgm mfklflyelqasf pudaoeuizoblaa yde tuiboszwbmbdjnjjdntpqeycnfkjuxttzhhzuanicbnklfeoslj ++
nwjfhkwhdruertkhqxtbbidzyfzabtbhbpgwj hxbdndzymze rdmwlslxmyrdjwjbedekuxbgwcqzwxsqhywpankwjsugprcmf  ++
kfiomiuqbtvpalbgaxeey tlymajo beuwlcjfurergqciweqpwyabyydpcxrwabzvhzrtczhywxxwkmtxwdzhsxuuvdjcumy yq ++
znarqxyzehdymwlrfwcxtzmwmntgtzigxjn lggyfmeglnad nwvzlnmizzvhwoedwurkqyergvx gcck ctt ichrlajotzzdqf ++


KeyboardInterrupt: 